# Here we want to place n events of transition between an ancestral lifestyle and a convergent lifestyle in a phylogeny. 
We want these n events to be independent, not nested.
We return them in a format compatible with Bio++ input files for bppseqgen and bppml.

## First, we generate a random topology:

In [1]:
from ete3 import Tree
import string
import scipy.stats as stats


t = Tree()
# We create a random tree topology
numTips = 20
candidateNames = list(string.ascii_lowercase)
tipNames = candidateNames[0:20]
t.populate(numTips, names_library=tipNames)
print (t)

#Alternatively we could read a tree from a file into a string "line", and then use: 
# t =  Tree( line )


            /-r
         /-|
        |   \-q
      /-|
     |  |   /-p
     |   \-|
     |      \-o
     |
   /-|      /-n
  |  |   /-|
  |  |  |   \-m
  |  |  |
  |  |  |      /-l
  |   \-|   /-|
  |     |  |   \-k
  |     |  |
  |     |  |      /-j
  |      \-|   /-|
  |        |  |  |   /-i
  |        |  |   \-|
  |        |  |     |   /-h
--|         \-|      \-|
  |           |         \-g
  |           |
  |           |   /-f
  |            \-|
  |               \-e
  |
  |         /-d
  |      /-|
  |     |   \-c
  |   /-|
  |  |  |   /-b
  |  |   \-|
   \-|      \-a
     |
     |   /-t
      \-|
         \-s


## Now, let's number the nodes of the tree.

In [2]:
nodeId = 0
for n in t.traverse():
    n.add_features(ND=nodeId)
    nodeId = nodeId + 1
#Writing in NHX format
t.write(features=['ND'])

'((((r:1[&&NHX:ND=15],q:1[&&NHX:ND=16])1:1[&&NHX:ND=7],(p:1[&&NHX:ND=17],o:1[&&NHX:ND=18])1:1[&&NHX:ND=8])1:1[&&NHX:ND=3],((n:1[&&NHX:ND=19],m:1[&&NHX:ND=20])1:1[&&NHX:ND=9],((l:1[&&NHX:ND=27],k:1[&&NHX:ND=28])1:1[&&NHX:ND=21],((j:1[&&NHX:ND=31],(i:1[&&NHX:ND=35],(h:1[&&NHX:ND=37],g:1[&&NHX:ND=38])1:1[&&NHX:ND=36])1:1[&&NHX:ND=32])1:1[&&NHX:ND=29],(f:1[&&NHX:ND=33],e:1[&&NHX:ND=34])1:1[&&NHX:ND=30])1:1[&&NHX:ND=22])1:1[&&NHX:ND=10])1:1[&&NHX:ND=4])1:1[&&NHX:ND=1],(((d:1[&&NHX:ND=23],c:1[&&NHX:ND=24])1:1[&&NHX:ND=11],(b:1[&&NHX:ND=25],a:1[&&NHX:ND=26])1:1[&&NHX:ND=12])1:1[&&NHX:ND=5],(t:1[&&NHX:ND=13],s:1[&&NHX:ND=14])1:1[&&NHX:ND=6])1:1[&&NHX:ND=2]);'

## Now we create a function to place the n events of transition.
We don't care about branch lengths, meaning that we decide that a transition is not more likely on a long branch than on a short one.

In [3]:
# We could use some dynamic programming to be able to generate paths that yield n transitions exactly.
# INstead we randomly generate transitions on the tree until we get the desired number.
# We have two states: ancestral (0) and convergent (1).
# We count the numbers of transitions
def randomTransitions(numTransitions, tree):
    numberOfNodes = len(tree.get_tree_root().get_descendants()) + 1
    rate = float(numTransitions)/float(numberOfNodes)
    ancestralTransition=dict()
    totalNumberOfTransitions = 0
    nodesWithTransitions = list()
    for node in tree.traverse("levelorder"):
        if node.is_root() :
            ancestralTransition[node] = False
        elif ( ancestralTransition[node.up] == True):
            ancestralTransition[node] = True
        else :
            sisterHasAlreadyTransitioned=False
            if ancestralTransition.__contains__(node.get_sisters()[0]): #Here we assume binary trees!
                sisterHasAlreadyTransitioned=True
            #randomly draw whether we do a transition or not
            transitionBool = stats.bernoulli.rvs(rate, size=1)[0] == 1
            if (transitionBool and not sisterHasAlreadyTransitioned):
                ancestralTransition[node] = True
                nodesWithTransitions.append(node)
                totalNumberOfTransitions = totalNumberOfTransitions + 1
            else:
                ancestralTransition[node] = False
    return nodesWithTransitions, totalNumberOfTransitions, ancestralTransition
        
        
        
def placeNTransitionsInTree(numTransitions, tree):
    observedNumTransitions = 2*numTransitions
    nodesWithTransitions = list()
    numTries = 0
    convergentModel = dict()
    while observedNumTransitions != numTransitions and numTries < 100:
        observedNumTransitions = 0
        nodesWithTransitions, observedNumTransitions, convergentModel = randomTransitions(numTransitions, tree)
        print ("Observed Number of Transitions: "+ str(observedNumTransitions ) + " compared to "+ str(numTransitions) + " wanted")
        numTries = numTries + 1
    if numTries < 100:
        for n in nodesWithTransitions:
            print(n.get_ascii())
    else:
        print("It seems like it is too difficult to place "+ str(numTransitions) + " events in this tree.")
    return convergentModel



## And now we place the transition events.

In [4]:
convergentModel = dict()
convergentModel = placeNTransitionsInTree(5, t)

Observed Number of Transitions: 4 compared to 5 wanted
Observed Number of Transitions: 2 compared to 5 wanted
Observed Number of Transitions: 3 compared to 5 wanted
Observed Number of Transitions: 1 compared to 5 wanted
Observed Number of Transitions: 2 compared to 5 wanted
Observed Number of Transitions: 2 compared to 5 wanted
Observed Number of Transitions: 0 compared to 5 wanted
Observed Number of Transitions: 2 compared to 5 wanted
Observed Number of Transitions: 1 compared to 5 wanted
Observed Number of Transitions: 3 compared to 5 wanted
Observed Number of Transitions: 2 compared to 5 wanted
Observed Number of Transitions: 1 compared to 5 wanted
Observed Number of Transitions: 5 compared to 5 wanted

   /-d
--|
   \-c

--t

--r

--n

--l


Now we want to output the tree and the command line for bppseqgen and bppml.

In [5]:
#First, get the nodes with the convergent model and the nodes with the ancestral model.
nodesWithConvergentModel = list()
nodesWithAncestralModel = list()
for k,v in convergentModel.items():
    if v == True:
        nodesWithConvergentModel.append(k.ND)
    if v == False:
        nodesWithAncestralModel.append(k.ND)


n1="\""+ str(nodesWithAncestralModel[0])
for n in nodesWithAncestralModel[1:len(nodesWithAncestralModel)-1]:
    n1 += "," + str(n) 
n1 += "\""
print(n1)
n2="\""+ str(nodesWithConvergentModel[0])
for n in nodesWithConvergentModel[1:len(nodesWithConvergentModel)-1]:
    n2 += "," + str(n) 
n2 += "\""
print(n2)
#Dummy values:
C1 = 1
C2 = 2
Nch = 1000
command="bppseqgen param=CATseq.bpp mod1Nodes=%s mod2Nodes=%s Nch=%d Ns1=%d Ns2=%d Ne1=%d Ne2=%d"%(n1,n2,Nch,C1,C2,C1,C2)
print (command)


"0,3,1,31,9,35,10,32,21,36,22,38,29,37,30,33,12,34,18,14,7,6,16,28,2,5,4,25,26,17,20"
"23,11,24,15,19,13"
bppseqgen param=CATseq.bpp mod1Nodes="0,3,1,31,9,35,10,32,21,36,22,38,29,37,30,33,12,34,18,14,7,6,16,28,2,5,4,25,26,17,20" mod2Nodes="23,11,24,15,19,13" Nch=1000 Ns1=1 Ns2=2 Ne1=1 Ne2=2
